В конце 2017 года платформа Civil Comments закрылась и опубликовала около 2 миллионов комментариев из социальных сетей, чтобы специалисты по данным со всего мира могли работать вместе над исследованием способов смягчения предвзятости в текстовых данных.

# Установка и импорт библиотек

In [3]:
!pip install numpy pandas --quiet
!pip install scikit-learn --quiet

In [4]:
# иморитирование всех необходимых библиотек
import pandas as pd
import numpy as np
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Загрузка данных

In [6]:
df = pd.read_csv('../data/toxic_comments/data_civil.csv')

In [7]:
df.head(5)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0000,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
1,239607,0.912500,Yet call out all Muslims for the acts of a few...,0.050000,0.237500,0.612500,0.887500,0.1125,0.0,0.0,...,26670,approved,0,0,0,1,0,0.000000,4,80
2,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.0000,0.0,0.0,...,26674,rejected,0,0,0,0,0,0.061538,4,65
3,240311,0.968750,You're an idiot.,0.031250,0.062500,0.000000,0.968750,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.000000,0,32
4,240329,0.900000,Who cares!? Stark trek and Star Wars fans are ...,0.100000,0.200000,0.000000,0.900000,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.300000,0,10


# Анализ данных

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90902 entries, 0 to 90901
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   90902 non-null  int64  
 1   target                               90902 non-null  float64
 2   comment_text                         90902 non-null  object 
 3   severe_toxicity                      90902 non-null  float64
 4   obscene                              90902 non-null  float64
 5   identity_attack                      90902 non-null  float64
 6   insult                               90902 non-null  float64
 7   threat                               90902 non-null  float64
 8   asian                                21687 non-null  float64
 9   atheist                              21687 non-null  float64
 10  bisexual                             21687 non-null  float64
 11  black                       

In [10]:
df.describe()

,id,target,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,...,parent_id,article_id,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
count,9.090200e+04,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,21687.00000,21687.000000,21687.000000,...,4.983000e+04,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000,90902.000000
mean,3.741905e+06,0.430254,0.020783,0.091598,0.054786,0.381360,0.025486,0.01059,0.003179,0.001616,...,3.683063e+06,281545.713736,0.258938,0.043365,0.106741,2.334360,0.559207,0.023667,1.535489,36.632560
std,2.450000e+06,0.406086,0.043101,0.182411,0.147375,0.386545,0.109004,0.08227,0.051270,0.024006,...,2.458295e+06,104028.940513,0.994396,0.240804,0.453165,4.587831,1.735908,0.106960,18.199186,147.288485
min,5.985600e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,2.210600e+05,2006.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,7.921310e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,7.825040e+05,159937.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
50%,5.232225e+06,0.500120,0.000000,0.000000,0.000000,0.200000,0.000000,0.00000,0.000000,0.000000,...,5.208236e+06,332720.500000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,6.000000
75%,5.785647e+06,0.810345,0.026316,0.116667,0.021739,0.785714,0.000000,0.00000,0.000000,0.000000,...,5.769276e+06,367055.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,58.000000
max,6.333872e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,6.333696e+06,399523.000000,58.000000,8.000000,16.000000,224.000000,96.000000,1.000000,1838.000000,4936.000000


In [11]:
def get_missing_stat(df): 
    missing_count = df.isna().sum()
    missing_percentage = (missing_count / len(df)) * 100
    missing_data = pd.DataFrame({
        'Missing Count': missing_count,
        'Missing Percentage': round(missing_percentage, 2)
    })
    return missing_data[missing_data['Missing Count'] > 0]

get_missing_stat(df)

,Missing Count,Missing Percentage
asian,69215,76.14
atheist,69215,76.14
bisexual,69215,76.14
black,69215,76.14
buddhist,69215,76.14
christian,69215,76.14
female,69215,76.14
heterosexual,69215,76.14
hindu,69215,76.14
homosexual_gay_or_lesbian,69215,76.14


Данные по признакам ориентации и религии сильно разряженны.

In [13]:
df[df['christian'] > 0].shape[0]

2436

In [14]:
df[df['muslim'] > 0].shape[0]

1680

In [15]:
df[df['buddhist'] > 0].shape[0]

57

Присутствует дизбаланс классов в выборке по религии.

# Построение baseline модели

Cделаем предпосылку, что комментарий считается токсичным, если вероятность выше 0.7

In [19]:
np.random.seed(0)
comments = df["comment_text"]
target = (df["target"] > 0.7).astype(int)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    comments,
    target,
    test_size=0.3,
    random_state=0
)

Преобразуем текст комментариев в числовой формат

In [22]:
vectorizer = CountVectorizer()

X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

Будем классифицировать комментарии на токсичные и нетоксичные с помощью логистической регрессии. Для оценки модели возьмем метрику accuracy и посчитаем ее.

In [24]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train_vectorized, y_train);

In [25]:
y_pred = model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {round(accuracy, 3)}")

Точность модели: 0.929


## Протестируем работу модели на примере нескольких комментариев

In [27]:
def predict_toxicity(comment):
    comment_vector = vectorizer.transform([comment])    
    probability = model.predict_proba(comment_vector)[0, 1]
    
    return round(probability, 3)

In [28]:
comment = "Apples are stupid"
toxicity_probability = predict_toxicity(comment)
print(f'Вероятность токсичности комментария "{comment}": {toxicity_probability:.2f}')

Вероятность токсичности комментария "Apples are stupid": 1.00


In [29]:
comment = "I love apples"
toxicity_probability = predict_toxicity(comment)
print(f'Вероятность токсичности комментария "{comment}": {toxicity_probability:.2f}')

Вероятность токсичности комментария "I love apples": 0.10


## Выведем наиболее токсичные слова

In [31]:
feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

In [32]:
word_coefficient_pairs = list(zip(feature_names, coefficients))
word_coefficient_pairs.sort(key=lambda x: abs(x[1]), reverse=True)

In [33]:
print("Наиболее значимые слова и их коэффициенты:")
for word, coef in word_coefficient_pairs[:10]:
    print(f"{word}: {coef:.4f}")

Наиболее значимые слова и их коэффициенты:
stupid: 9.5244
idiot: 8.6641
idiots: 8.6258
stupidity: 7.5645
idiotic: 6.8268
crap: 6.5549
dumb: 6.5321
pathetic: 6.4545
morons: 6.3688
moron: 6.3512


Похоже что эти слова используются в оскорблениях. Удивляет что idiot имеет более высокий коэффициент, чем, например, pathetic. Вероятно, оно встречается чаще и поэтому модель лучше обучилось на нём.

## Протестируем модель на ее предвзятость к религии

In [36]:
comments = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend",
]
for comment in comments:
    toxicity_probability = predict_toxicity(comment)
    print(f'Вероятность токсичности комментария "{comment}": {toxicity_probability:.2f}')

Вероятность токсичности комментария "I have a christian friend": 0.12
Вероятность токсичности комментария "I have a muslim friend": 0.45
Вероятность токсичности комментария "I have a white friend": 0.33
Вероятность токсичности комментария "I have a black friend": 0.52


У модели явно есть смещение(bias) относительно мусульман и афроамериканцев. В иделе мы должны были получить одинаково нетоксичную вероятность для христиан и мусульман, для белых и черных.

Модель проявляет исламофобию, это может указывать на нарушение демографического паритета, так как модель может давать больше положительных предсказаний (т.е. классифицировать как токсичные) для комментариев, связанных с исламом, чем для других религий. То есть истинно положительный показатель для мусульманских комментариев может быть завышен), а для других религий - занижен.

# Улучшение алгорима с точки зрения этики

Можно попытаться использовать дополнительные признаки("atheist", "buddhist", "christian", "hindu", "jewish", "latino", "muslim", "other_religion") чтобы попытаться уменьшить предвзятость модели. Это может помочь модели учитывать контекст, связанный с исламом, и уменьшить предвзятость.
Я попробовал и вот какой результат получился:  
**точность модели упала** 0.929 -> 0.928, а **модель стала еще менее этичной**:  
Вероятность токсичности комментария "I have a christian friend": 0.12 -> 0.21  
Вероятность токсичности комментария "I have a muslim friend": 0.45 -> 0.71  
Вероятность токсичности комментария "I have a white friend": 0.33 -> 0.39  
Вероятность токсичности комментария "I have a black friend": 0.52 -> 0.59  

Тогда можно попробовать использовать регуляризацию, чтобы уменьшить влияние отдельных признаков. И вот это уже принесло плоды
**точность модели упала** 0.929 -> 0.919, а **но модель стала более этичная**:  
Вероятность токсичности комментария "I have a christian friend": 0.12 -> 0.24  
Вероятность токсичности комментария "I have a muslim friend": 0.45 -> 0.43  
Вероятность токсичности комментария "I have a white friend": 0.33 -> 0.40  
Вероятность токсичности комментария "I have a black friend": 0.52 -> 0.49  

## Эксперемент с использованием признаков религии

In [42]:
documents = df["comment_text"]
labels = (df["target"] > 0.7).astype(int)
columns_to_fill = [
    "atheist",
    "buddhist",
    "christian",
    "hindu",
    "jewish",
    "latino",
    "muslim",
    "other_religion"
]
df[columns_to_fill] = df[columns_to_fill].fillna(0)
religion_features = df[columns_to_fill].to_numpy()


vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(documents)
X = hstack([X_text, religion_features])
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {round(accuracy, 3)}")

Точность модели: 0.928


In [43]:
comments = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend",
]
X_new_text = vectorizer.transform(comments)

new_atheist_religion_feature = np.array([0.0, 0.0, 0.0, 0.0]).reshape(-1, 1)
new_buddhist_religion_feature = np.array([0.0, 0.0, 0.0, 0.0]).reshape(-1, 1)
new_christian_religion_feature = np.array([1.0, 0.0, 0.0, 0.0]).reshape(-1, 1)
new_hindu_religion_feature = np.array([0.0, 0.0, 0.0, 0.0]).reshape(-1, 1)
new_jewish_religion_feature = np.array([0.0, 0.0, 0.0, 0.0]).reshape(-1, 1)
new_latino_religion_feature = np.array([0.0, 0.0, 0.0, 0.0]).reshape(-1, 1)
new_muslim_feature = np.array([0.0, 1.0, 0.0, 0.0]).reshape(-1, 1)
new_other_religion_feature = np.array([0.0, 0.0, 0.0, 0.0]).reshape(-1, 1)

X_new = hstack([
    X_new_text,
    new_atheist_religion_feature,
    new_buddhist_religion_feature,
    new_christian_religion_feature,
    new_hindu_religion_feature,
    new_jewish_religion_feature,
    new_latino_religion_feature,
    new_muslim_feature,
    new_other_religion_feature,
])
predictions = model.predict(X_new)
probabilities = model.predict_proba(X_new)[:, 1]

for comment, prediction, probability in zip(comments, predictions, probabilities):
    print(f'Вероятность токсичности комментария: "{comment}": {probability:.2f}')

Вероятность токсичности комментария: "I have a christian friend": 0.21
Вероятность токсичности комментария: "I have a muslim friend": 0.71
Вероятность токсичности комментария: "I have a white friend": 0.39
Вероятность токсичности комментария: "I have a black friend": 0.59


## Эксперемент с регулязацией

In [45]:
comments = df["comment_text"]
y = (df["target"] > 0.7).astype(int)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(comments)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0
)

In [46]:
model = LogisticRegression(max_iter=2000, C=0.1, penalty='l2')
model.fit(X_train, y_train);

In [47]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {round(accuracy, 3)}")

Точность модели: 0.919


In [48]:
comments = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend",
]
for comment in comments:
    toxicity_probability = predict_toxicity(comment)
    print(f'Вероятность токсичности комментария "{comment}": {toxicity_probability:.2f}')

Вероятность токсичности комментария "I have a christian friend": 0.24
Вероятность токсичности комментария "I have a muslim friend": 0.43
Вероятность токсичности комментария "I have a white friend": 0.40
Вероятность токсичности комментария "I have a black friend": 0.49
